# Relaxation effects ($ T_2 $)

In this notebook we demonstrate the use of additional $T_2$ relaxation effects applied on top of an existing TissueModel.

## Setting up the dmipy model

We demonstrate this using a simple model from dmipy

In [1]:
from dmipy.core.modeling_framework import MultiCompartmentModel
from dmipy.signal_models.gaussian_models import G1Ball

from microtool.dmipy import DmipyTissueModel
from microtool.tissue_model import RelaxedMultiTissueModel

# setting up a simple model
simple_ball = G1Ball(lambda_iso=1.7e-9)

# This step is unnecessary
dmipy_model = MultiCompartmentModel([simple_ball])

microtool_model = DmipyTissueModel(dmipy_model)

relaxation_time = 100.0
model = RelaxedMultiTissueModel([microtool_model],[1.0], [relaxation_time])
print(model)

Tissue model with 3 scalar parameters:
Tissue-parameter                 Value    Scale  Optimize    Fit    Fit Bounds
---------------------------  ---------  -------  ----------  -----  ------------------------------------------------
model_0_G1Ball_1_lambda_iso    1.7e-09    1e-09  True        True   (1.0000000000000002e-10, 3.0000000000000004e-09)
T2_relaxation_0              100          1      True        True   (0.1, 1000.0)
S0                             1          1      False       False  (0.0, 2.0)


## Setting up the acquisition scheme

In [2]:
from microtool.dmipy import convert_dmipy_scheme2diffusion_scheme

from microtool.utils.saved_schemes import alexander2008_optimized_directions


acq_scheme = alexander2008_optimized_directions([2,8,16])
acq_wrapped = convert_dmipy_scheme2diffusion_scheme(acq_scheme)

print(acq_wrapped)

Acquisition scheme with 44 measurements and 6 scalar parameters:
  DiffusionBValue [s/mm²] in (0.0, 20000.0)    DiffusionGradientAnglePhi [rad] (fixed)    DiffusionGradientAngleTheta [rad] (fixed)    DiffusionPulseWidth [ms] in (1.0, 100.0)    DiffusionPulseInterval [ms] in (1.0, 1000.0)    EchoTime [ms] in (0.1, 1000.0)
-------------------------------------------  -----------------------------------------  -------------------------------------------  ------------------------------------------  ----------------------------------------------  --------------------------------
                                          0                                 -1.96142                                       1.98074                                            7                                              12                           36.2367
                                          0                                  2.96645                                       0.952638                              

## Generating signal

In [3]:
# relaxation signal
model(acq_wrapped)

array([6.96027114e-01, 6.96027114e-01, 6.96027114e-01, 6.96027114e-01,
       6.96027114e-01, 6.96027114e-01, 6.96027114e-01, 6.96027114e-01,
       6.96027114e-01, 6.96027114e-01, 6.96027114e-01, 6.96027114e-01,
       6.96027114e-01, 6.96027114e-01, 6.96027114e-01, 6.96027114e-01,
       6.96027114e-01, 6.96027114e-01, 8.20903969e-02, 8.20903969e-02,
       1.39291019e-03, 1.39291019e-03, 1.39291019e-03, 1.39291019e-03,
       1.39291019e-03, 1.39291019e-03, 1.39291019e-03, 1.39291019e-03,
       7.92122882e-14, 7.92122882e-14, 7.92122882e-14, 7.92122882e-14,
       7.92122882e-14, 7.92122882e-14, 7.92122882e-14, 7.92122882e-14,
       7.92122882e-14, 7.92122882e-14, 7.92122882e-14, 7.92122882e-14,
       7.92122882e-14, 7.92122882e-14, 7.92122882e-14, 7.92122882e-14])

## Optimizing the scheme with respect to $ T_2 $.

In [ ]:
from microtool.optimize import optimize_scheme

scheme, _ = optimize_scheme(acq_wrapped, model, noise_variance=0.02, method='differential_evolution')

differential_evolution step 1: f(x)= 0.0738211
differential_evolution step 2: f(x)= 0.0615543
differential_evolution step 3: f(x)= 0.0615543
differential_evolution step 4: f(x)= 0.0615543
differential_evolution step 5: f(x)= 0.0478345
differential_evolution step 6: f(x)= 0.0478345
differential_evolution step 7: f(x)= 0.0478345
differential_evolution step 8: f(x)= 0.0478345
differential_evolution step 9: f(x)= 0.0478345
differential_evolution step 10: f(x)= 0.0478345
differential_evolution step 11: f(x)= 0.0478345
differential_evolution step 12: f(x)= 0.0478345
differential_evolution step 13: f(x)= 0.0332748
differential_evolution step 14: f(x)= 0.0303993
differential_evolution step 15: f(x)= 0.0303993
differential_evolution step 16: f(x)= 0.0303993
differential_evolution step 17: f(x)= 0.0303993
differential_evolution step 18: f(x)= 0.0303993
differential_evolution step 19: f(x)= 0.0303993
differential_evolution step 20: f(x)= 0.0303993
differential_evolution step 21: f(x)= 0.0303993
d

## Fitting the model

## Validation of scheme optimization through Monte Carlo simulations

In [ ]:
from microtool.monte_carlo.simulation import MonteCarloSimulation
from scipy import stats

# loc is mean and scale is standard deviation
noise_distribution = stats.norm(loc=0, scale=0.02)

simulation = MonteCarloSimulation(scheme,model,noise_distribution,n_sim=100)

simulation.run()
simulation.save